In [1]:
import torch
import tensorflow as tf
import tensorflow.keras as keras
from x_transformers import Encoder as PT_Encoder
from tf_x_transformers import Encoder as TF_Encoder
pt_x = torch.randn(12,128,768)
tf_x = tf.constant(pt_x.numpy())


In [2]:
ndepths = 6
rel_pos_bias = True
rotary_pos_emb = True
gate_residual = True
pre_norm = False
macaron = False
use_scalenorm = use_rmsnorm = use_rezero = False
pt_model = PT_Encoder(
    dim=768,
    depth=ndepths,
    heads=12,
    rel_pos_bias=rel_pos_bias,
    rotary_pos_emb=rotary_pos_emb,
    attn_dim_head=64,
    gate_residual=gate_residual,
    pre_norm=pre_norm,
)
pt_model.eval()
class TMP(keras.Model):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self.model = TF_Encoder(
            dim=768,
            depth=ndepths,
            heads=12,
            rel_pos_bias=rel_pos_bias,
            rotary_pos_emb=rotary_pos_emb,
            attn_dim_head=64,
            gate_residual=gate_residual,
            pre_norm=pre_norm
        )
    def call(self,x):
        return self.model(x)
tf.keras.backend.set_learning_phase(0)
tf_model = TMP()
dummp = tf_model(tf_x)

C:\Users\yujun\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\keras\backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [3]:
tf_path = "./ckpts/checkpoint"
tf_model.save_weights(tf_path)  
init_vars = tf.train.list_variables(tf_path)
names = []
arrays = []
layer_depth = []
for full_name, shape in init_vars:
    if full_name == "_CHECKPOINTABLE_OBJECT_GRAPH" or full_name[0] in ["global_step", "save_counter"]:
        print(f"Skipping non-model layer {full_name}")
        continue
    names.append(full_name.replace(".ATTRIBUTES/VARIABLE_VALUE",""))
    array = tf.train.load_variable(tf_path, full_name)
    array = torch.from_numpy(array)
    if "kernel" in names[-1]:
        array = array.T
    arrays.append(array)
mmap = dict(zip(names,arrays))

Skipping non-model layer _CHECKPOINTABLE_OBJECT_GRAPH


In [4]:
names

['model/rel_pos/relative_attention_bias/embeddings/',
 'model/tf_layers/0/0/beta/',
 'model/tf_layers/0/0/gamma/',
 'model/tf_layers/0/1/to_k/kernel/',
 'model/tf_layers/0/1/to_out/bias/',
 'model/tf_layers/0/1/to_out/kernel/',
 'model/tf_layers/0/1/to_q/kernel/',
 'model/tf_layers/0/1/to_v/kernel/',
 'model/tf_layers/0/2/gru/bias/',
 'model/tf_layers/0/2/gru/kernel/',
 'model/tf_layers/0/2/gru/recurrent_kernel/',
 'model/tf_layers/1/0/beta/',
 'model/tf_layers/1/0/gamma/',
 'model/tf_layers/1/1/net/layer_with_weights-0/bias/',
 'model/tf_layers/1/1/net/layer_with_weights-0/kernel/',
 'model/tf_layers/1/1/net/layer_with_weights-1/bias/',
 'model/tf_layers/1/1/net/layer_with_weights-1/kernel/',
 'model/tf_layers/1/2/gru/bias/',
 'model/tf_layers/1/2/gru/kernel/',
 'model/tf_layers/1/2/gru/recurrent_kernel/',
 'model/tf_layers/10/0/beta/',
 'model/tf_layers/10/0/gamma/',
 'model/tf_layers/10/1/to_k/kernel/',
 'model/tf_layers/10/1/to_out/bias/',
 'model/tf_layers/10/1/to_out/kernel/',
 '

In [5]:
for k,v in pt_model.state_dict().items():
    print(k,v.shape)

layers.0.0.weight torch.Size([768])
layers.0.0.bias torch.Size([768])
layers.0.1.to_q.weight torch.Size([768, 768])
layers.0.1.to_k.weight torch.Size([768, 768])
layers.0.1.to_v.weight torch.Size([768, 768])
layers.0.1.to_out.weight torch.Size([768, 768])
layers.0.1.to_out.bias torch.Size([768])
layers.0.2.gru.weight_ih torch.Size([2304, 768])
layers.0.2.gru.weight_hh torch.Size([2304, 768])
layers.0.2.gru.bias_ih torch.Size([2304])
layers.0.2.gru.bias_hh torch.Size([2304])
layers.1.0.weight torch.Size([768])
layers.1.0.bias torch.Size([768])
layers.1.1.net.0.0.weight torch.Size([3072, 768])
layers.1.1.net.0.0.bias torch.Size([3072])
layers.1.1.net.2.weight torch.Size([768, 3072])
layers.1.1.net.2.bias torch.Size([768])
layers.1.2.gru.weight_ih torch.Size([2304, 768])
layers.1.2.gru.weight_hh torch.Size([2304, 768])
layers.1.2.gru.bias_ih torch.Size([2304])
layers.1.2.gru.bias_hh torch.Size([2304])
layers.2.0.weight torch.Size([768])
layers.2.0.bias torch.Size([768])
layers.2.1.to_q.we

In [6]:
def reorder(x):
    z,r,h= x.chunk(3,dim=0)
    return torch.cat([r,z,h],dim=0)

# rel_pos_bias
if rel_pos_bias and 'model/rel_pos/relative_attention_bias/embeddings/' in mmap:
    pt_model.rel_pos.relative_attention_bias.weight.data = mmap['model/rel_pos/relative_attention_bias/embeddings/']

for i,layer in enumerate(pt_model.layer_types):
    is_last = i==len(pt_model.layer_types)-1
    i = str(i)

    if layer == "f" :
        if macaron:
            getattr(getattr(getattr(getattr(pt_model.layers,i),"1").fn.net,"0"),"0").weight.data = mmap[f"model/tf_layers/{i}/1/fn/net/layer_with_weights-0/kernel/"]
            getattr(getattr(getattr(getattr(pt_model.layers,i),"1").fn.net,"0"),"0").bias.data = mmap[f"model/tf_layers/{i}/1/fn/net/layer_with_weights-0/bias/"]
            getattr(getattr(getattr(pt_model.layers,i),"1").fn.net,"2").weight.data = mmap[f"model/tf_layers/{i}/1/fn/net/layer_with_weights-1/kernel/"]
            getattr(getattr(getattr(pt_model.layers,i),"1").fn.net,"2").bias.data = mmap[f"model/tf_layers/{i}/1/fn/net/layer_with_weights-1/bias/"]
        else:
            if not is_last:
                if use_scalenorm or use_rmsnorm:
                    getattr(getattr(pt_model.layers,i),"0").g.data = mmap[f"model/tf_layers/{i}/0/g/"]

                else:
                    getattr(getattr(pt_model.layers,i),"0").weight.data = mmap[f"model/tf_layers/{i}/0/gamma/"]
                    getattr(getattr(pt_model.layers,i),"0").bias.data = mmap[f"model/tf_layers/{i}/0/beta/"]

            getattr(getattr(getattr(getattr(pt_model.layers,i),"1").net,"0"),"0").weight.data = mmap[f"model/tf_layers/{i}/1/net/layer_with_weights-0/kernel/"]
            getattr(getattr(getattr(getattr(pt_model.layers,i),"1").net,"0"),"0").bias.data = mmap[f"model/tf_layers/{i}/1/net/layer_with_weights-0/bias/"]
            getattr(getattr(getattr(pt_model.layers,i),"1").net,"2").weight.data = mmap[f"model/tf_layers/{i}/1/net/layer_with_weights-1/kernel/"]
            getattr(getattr(getattr(pt_model.layers,i),"1").net,"2").bias.data = mmap[f"model/tf_layers/{i}/1/net/layer_with_weights-1/bias/"]

    elif layer =="a" or layer=="c":
        if use_scalenorm or use_rmsnorm or use_rezero:
            getattr(getattr(pt_model.layers,i),"0").g.data = mmap[f"model/tf_layers/{i}/0/g/"]
        else:
            getattr(getattr(pt_model.layers,i),"0").weight.data = mmap[f"model/tf_layers/{i}/0/gamma/"]
            getattr(getattr(pt_model.layers,i),"0").bias.data = mmap[f"model/tf_layers/{i}/0/beta/"]

        try:
            getattr(getattr(pt_model.layers,i),"1").post_softmax_proj.data = mmap[f"model/tf_layers/{i}/1/post_softmax_proj/"]
            getattr(getattr(pt_model.layers,i),"1").pre_softmax_proj.data = mmap[f"model/tf_layers/{i}/1/pre_softmax_proj/"]
        except:
            pass

        getattr(getattr(pt_model.layers,i),"1").to_q.weight.data = mmap[f"model/tf_layers/{i}/1/to_q/kernel/"]
        getattr(getattr(pt_model.layers,i),"1").to_k.weight.data = mmap[f"model/tf_layers/{i}/1/to_k/kernel/"]
        getattr(getattr(pt_model.layers,i),"1").to_v.weight.data = mmap[f"model/tf_layers/{i}/1/to_v/kernel/"]
        getattr(getattr(pt_model.layers,i),"1").to_out.weight.data = mmap[f"model/tf_layers/{i}/1/to_out/kernel/"]
        getattr(getattr(pt_model.layers,i),"1").to_out.bias.data = mmap[f"model/tf_layers/{i}/1/to_out/bias/"]

    # gate_residual
    if gate_residual:
        '''
        pt gru weights format is r,z,h
        tf gru weights format is z,r,h
        '''
        getattr(getattr(pt_model.layers,i),"2").gru.weight_ih.data = reorder(mmap[f"model/tf_layers/{i}/2/gru/kernel/"])
        getattr(getattr(pt_model.layers,i),"2").gru.weight_hh.data = reorder(mmap[f"model/tf_layers/{i}/2/gru/recurrent_kernel/"])
        ih,hh = mmap[f"model/tf_layers/{i}/2/gru/bias/"].unbind(0)
        getattr(getattr(pt_model.layers,i),"2").gru.bias_ih.data = reorder(ih)
        getattr(getattr(pt_model.layers,i),"2").gru.bias_hh.data = reorder(hh)


In [7]:
with torch.no_grad():
    pt_outputs = pt_model(pt_x)
tf_outputs = tf_model(tf_x)
tf_outputs = torch.from_numpy(tf_outputs.numpy())

In [8]:
print("mean difference",(pt_outputs-tf_outputs).abs().mean())

mean difference tensor(5.0120e-07)


In [9]:
print("max difference",(pt_outputs-tf_outputs).abs().max())

max difference tensor(1.3351e-05)
